# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from z3 import *
from itertools import combinations
import sys
sys.path.append('../../scripts/biota/')
from BIoTA_Control import *

# Fixed Parameter

In [5]:
CO2_FRESH_AIR = 400             # CO2 concentration (ppm) of fresh air
TEMP_FRESH_AIR = 91.4           # Temperature (33 degree F) of the fresh air
CP_AIR = 1.026                  # Specific heat of fresh air
DEF_TEMP_SUPPLY_AIR =  55.4     # Default temperature (degree fahrenheit) of supply air (13 degree celsius)
COST_PER_KWH = 0.1137           # USD Cost per KWh

# Variable Parameters

In [6]:
zone_temp_setpoint = [0, 75.2, 75.2, 75.2, 75.2]     # list of temperature (fahrenheit) setpoint of the different zones
zone_co2_setpoint = [0, 1000, 1000, 1000, 1000]      # list of CO2 (ppm) setpoint of the corresponding zones
control_time = 1                                     # control time (in minute)

# Generating Possible Combination of Occupants

In [8]:
import itertools
all_samples = list(itertools.permutations([1, 1, 0, 0, 0]))
all_samples += list(itertools.permutations([2, 0, 0, 0, 0]))
# however, there are repetations

unique_samples = set()

for data in all_samples:    
    unique_samples.add(data)
    
unique_samples

{(0, 0, 0, 0, 2),
 (0, 0, 0, 1, 1),
 (0, 0, 0, 2, 0),
 (0, 0, 1, 0, 1),
 (0, 0, 1, 1, 0),
 (0, 0, 2, 0, 0),
 (0, 1, 0, 0, 1),
 (0, 1, 0, 1, 0),
 (0, 1, 1, 0, 0),
 (0, 2, 0, 0, 0),
 (1, 0, 0, 0, 1),
 (1, 0, 0, 1, 0),
 (1, 0, 1, 0, 0),
 (1, 1, 0, 0, 0),
 (2, 0, 0, 0, 0)}

# Tabulate the Control Costs for all Possible Combinations

In [9]:
# convert list to a tuple
def to_tuple(a):
    try:
        return tuple(to_tuple(i) for i in a)
    except TypeError:
        return a
    
dict_control_cost = dict()
for sample in unique_samples:
    zone_occupant = list(sample)    
    dict_control_cost[sample] = control_cost(zones, zone_occupant, zone_temp_setpoint, zone_volume, pp_co2, pp_heat, load, zone_co2_setpoint, control_time)
dict_control_cost

{(1, 0, 1, 0, 0): 0.0007955147465,
 (1, 1, 0, 0, 0): 0.0007418396295,
 (0, 0, 1, 0, 1): 0.00420112231555,
 (2, 0, 0, 0, 0): 0.0,
 (0, 0, 0, 2, 0): 0.0127561109392,
 (0, 0, 0, 0, 2): 0.0068112151381,
 (0, 1, 0, 0, 1): 0.00414744719855,
 (1, 0, 0, 0, 1): 0.00340560756905,
 (0, 0, 1, 1, 0): 0.0071735702161000005,
 (0, 2, 0, 0, 0): 0.001483679259,
 (0, 0, 0, 1, 1): 0.00978366303865,
 (0, 1, 0, 1, 0): 0.0071198950991,
 (0, 0, 2, 0, 0): 0.001591029493,
 (0, 1, 1, 0, 0): 0.0015373543760000001,
 (1, 0, 0, 1, 0): 0.0063780554696}

# Reading SHATTER Attack-Schedule Datasets

In [14]:
house_A_occ_1_final_schedule = pd.read_csv("../../data/shatter/House-A_Occ-1_SHATTER_Schedule.csv").values
house_A_occ_2_final_schedule = pd.read_csv("../../data/shatter/House-A_Occ-2_SHATTER_Schedule.csv").values
house_B_occ_1_final_schedule = pd.read_csv("../../data/shatter/House-B_Occ-1_SHATTER_Schedule.csv").values
house_B_occ_2_final_schedule = pd.read_csv("../../data/shatter/House-B_Occ-2_SHATTER_Schedule.csv").values
house_A_occ_1_greedy_schedule =  pd.read_csv("../../data/shatter/House-A_Occ-1_Greedy_Schedule.csv").values
house_A_occ_2_greedy_schedule =  pd.read_csv("../../data/shatter/House-A_Occ-2_Greedy_Schedule.csv").values
house_B_occ_1_greedy_schedule =  pd.read_csv("../../data/shatter/House-B_Occ-1_Greedy_Schedule.csv").values
house_B_occ_2_greedy_schedule =  pd.read_csv("../../data/shatter/House-B_Occ-2_Greedy_Schedule.csv").values

# SHATTER Attack Cost Calculation for House-A

In [37]:
shatter_cost_house_A = []

for i in range(len(house_A_occ_1_final_schedule)):
    zone_occupants = [0, 0, 0, 0, 0]
    zone_occupants[int(house_A_occ_1_final_schedule[i])] += 1
    zone_occupants[int(house_A_occ_2_final_schedule[i])] += 1
    shatter_cost_house_A.append(dict_control_cost[to_tuple(zone_occupants)])  
sum(shatter_cost_house_A) * 30

232.0873550428728

# SHATTER Control Cost Calculation for House-B

In [36]:
shatter_cost_house_B = []

for i in range(len(house_B_occ_1_final_schedule)):
    zone_occupants = [0, 0, 0, 0, 0]
    zone_occupants[int(house_B_occ_1_final_schedule[i])] += 1
    zone_occupants[int(house_B_occ_2_final_schedule[i])] += 1
    shatter_cost_house_B.append(dict_control_cost[to_tuple(zone_occupants)])           
sum(shatter_cost_house_B) * 30

196.0337037300551

# Greedy Attack Cost Calculation for House-A

In [35]:
greedy_cost_house_A = []

for i in range(len(house_A_occ_1_greedy_schedule)):
    zone_occupants = [0, 0, 0, 0, 0]
    zone_occupants[int(house_A_occ_1_greedy_schedule[i])] += 1
    zone_occupants[int(house_A_occ_2_greedy_schedule[i])] += 1
    greedy_cost_house_A.append(dict_control_cost[to_tuple(zone_occupants)])  
sum(greedy_cost_house_A) * 30

201.41906050899362

# Greedy Attack Cost Calculation for House-B

In [34]:
greedy_cost_house_B = []

for i in range(len(house_B_occ_1_greedy_schedule)):
    zone_occupants = [0, 0, 0, 0, 0]
    zone_occupants[int(house_B_occ_1_greedy_schedule[i])] += 1
    zone_occupants[int(house_B_occ_2_greedy_schedule[i])] += 1
    greedy_cost_house_B.append(dict_control_cost[to_tuple(zone_occupants)])  
sum(greedy_cost_house_B) * 30

195.37919577664974

# Reading BIoTA Attack Dataset

In [28]:
biota_house_A = pd.read_csv("../../data/biota/BIoTA-Attack-Dataframe_House-A.csv")
biota_house_B = pd.read_csv("../../data/biota/BIoTA-Attack-Dataframe_House-B.csv")

# Day-wise BIoTA Attack Cost House-A

In [29]:
day_wise_biota_attack_cost_house_A = []
for i in range(0, 43200, 1440):
    day_wise_biota_attack_cost_house_A.append(sum(biota_house_A["Attack Control Cost ($)"][i:i+1400]))
total_biota_attack_cost_house_A = sum(day_wise_biota_attack_cost_house_A)
total_biota_attack_cost_house_A

535.7566594464079

# Day-wise BIoTA Attack Cost House-B

In [30]:
day_wise_biota_attack_cost_house_B = []
for i in range(0, 43200, 1440):
    day_wise_biota_attack_cost_house_B.append(sum(biota_house_B["Attack Control Cost ($)"][i:i+1400]))
total_biota_attack_cost_house_B = sum(day_wise_biota_attack_cost_house_B)
total_biota_attack_cost_house_B

535.7566594464079